# Microsoft GraphRAG Demo

GraphRAG is a knowledge graph-based approach to Retrieval Augmented Generation (RAG) that provides LLMs with complete understanding of a dataset at global and local levels. It does this by constructing a graph of entities & relationships which are then partitioned into hierarchical communities. At query time, LLM context is built using a combination of entity description vector search, graph traversals, community summaries and raw text. GraphRAG has two main search methods, Global Search & Local Search, where local is more entity driven and global relies more heavily on communities.

To learn more check out the Microsoft [blog post](https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/) and GraphRAG [paper](https://arxiv.org/pdf/2404.16130)

In [1]:
import os
import pandas as pd
import warnings
from IPython.display import Markdown
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from src.doc_store.cl_audio_client import CourtListenerAudio
from src.graphrag.graphrag_manager import GraphRAGManager

In [3]:
cl_audio = CourtListenerAudio()

In [4]:
search_res = cl_audio.search_oral_arguments('"Ryan Morgan"')
len(search_res)

3

In [15]:
res_dicts = [s.model_dump() for s in search_res]
res_df = pd.DataFrame(res_dicts)
res_df['timestamp'] = pd.to_datetime(res_df['timestamp']).dt.date
Markdown(res_df[['timestamp', 'case_name', 'court', 'frontend_url']].to_markdown(index=False))

| timestamp   | case_name                            | court                                     | frontend_url                                                                  |
|:------------|:-------------------------------------|:------------------------------------------|:------------------------------------------------------------------------------|
| 2024-06-22  | Curtis Hamrick v. Partsfleet, LLC    | Court of Appeals for the Eleventh Circuit | https://www.courtlistener.com/audio/70619/curtis-hamrick-v-partsfleet-llc/    |
| 2024-06-23  | Bruce Gates v. TF Final Mile, LLC    | Court of Appeals for the Eleventh Circuit | https://www.courtlistener.com/audio/64607/bruce-gates-v-tf-final-mile-llc/    |
| 2024-06-21  | Anthony Wright v. Waste Pro USA Inc. | Court of Appeals for the Eleventh Circuit | https://www.courtlistener.com/audio/86814/anthony-wright-v-waste-pro-usa-inc/ |

In [6]:
output_directory = "src/graphrag/input"
cl_audio.save_texts_to_files(search_res, output_directory)

Saved snippet to src/graphrag/input\curtis_hamrick_v._partsfleet_llc_cl.txt
Saved snippet to src/graphrag/input\bruce_gates_v._tf_final_mile_llc_cl.txt
Saved snippet to src/graphrag/input\anthony_wright_v._waste_pro_usa_inc._cl.txt


In [8]:
Markdown(str(search_res[0])[:4000])

Case ID: 19-13339

Source: [court website](https://www.courtlistener.com/audio/70619/curtis-hamrick-v-partsfleet-llc/)

Curtis Hamrick v. Partsfleet, LLC

Date Argued: 2020-06-12

Court: Court of Appeals for the Eleventh Circuit

Curtis Hamrick v. Partsfleet, LLC
12 June 2020
19-13339
    The next case up is Hamrick v. Partsfleet and in the Hamrick case we&#x27;ll hear first from Mr. Magnus. Thank you, Your Honor. May it please the Court, Eric Magnus for the appellant&#x27;s Partsfleet LLC et al. The district court erred in denying appellant&#x27;s motion to compel arbitration. An application of the facts demonstrates that plaintiffs are not transportation workers for purposes of the FAA under this court&#x27;s decision in Hill v. Renner Center or by applying the factors laid out by the 8th Circuit in Lenz v. Yellow Transportation. The lower court found dispositive whether the goods the drivers transported came from out of state were delivered untransformed. If the lower court was correct that this fact is dispositive, then Hill was wrongly decided. Of course, Hill was correctly decided in his binding precedent requiring reversal. Concluding that plaintiffs are transportation workers is also inconsistent with the limiting language in the statute restricting transportation workers to those similar to railroad workers or seamen and Congress&#x27;s intent and purpose in adopting the limited exception to the FAA. Finally, even if the transportation worker exception applies, the district court erred by not deciding whether arbitration should be compelled pursuant to state law because the applicable state arbitration acts apply to the parties&#x27; arbitration agreements. First, comparing the facts of this case to Hill is telling. Hill was the unanimous decision in which Your Honor, Judge Carnes was on the panel, which held that Hill was not a transportation worker. Plaintiffs, in this case, were less of transportation workers than Hill was. First, two critical facts are the same. Some of the products both delivered had come from out of state, untransformed. The lower court considered this to be a difference, in fact, a dispositive difference, but it&#x27;s not. If the issue of whether the products came from out of state is outcome determinative, then Hill was wrongly decided because the furniture and appliances that Hill delivered were not built in the states where Hill delivered them. Rent-A-Center is a national company, and there was no evidence in the record that the furniture and electronics were all made in Georgia, Alabama, or even in the United States. Counsel, this is Robert Luck. Is the test in Hill that was adopted from Palandino that the goods themselves that are being transported must have been in interstate commerce and not have been disassembled and then reassembled, or is the test that the worker him or herself must travel in interstate commerce? Your Honor, the latter is the test that was adopted in Hill. The former is the test that was effectively applied by the lower court, and that&#x27;s precisely... Why is that not the beginning and the end? If that is the test, why is that not the beginning of the end of why this is wrong, at least as to the vast majority of the plaintiffs&#x27; collective action where they did not travel, in other words, the 99% that didn&#x27;t travel interstate? Well, because, Your Honor, traveling in interstate commerce is not the only... It cannot merely be that traveling in interstate commerce makes someone a transportation worker or... Agreed. They have to have... The transportation can&#x27;t be incidental, right? I have to say, just reading the record, you&#x27;re going to have a tougher time convincing me, and I&#x27;m only speaking for myself, that these folks who are in the delivery business and your client is in the delivery business were not transportation workers in that sense, in the non-incidental sense, like the customer service rep in Hill and the customer service rep in Pal

In [48]:
import subprocess

def run_graphrag_cli(
    venv_path: str,
    root: str,
    config: str = None,
    verbose: bool = True,
    resume: str = None,
    reporter: str = "print",
    emit: str = "parquet",
    nocache: bool = False
) -> subprocess.CompletedProcess:
    """
    Execute the GraphRAG CLI command with the given parameters using a specific virtual environment.

    Args:
        venv_path (str): Path to the virtual environment directory.
        root (str): The workspace project root.
        config (str): Path to the custom config YAML file.
        verbose (bool): Whether to run in verbose mode. Defaults to True.
        resume (str): Timestamp to resume from. Defaults to None.
        reporter (str): Reporter type ('rich', 'print', or 'none'). Defaults to "print".
        emit (str): Output format(s), comma-separated. Defaults to "parquet".
        nocache (bool): Whether to disable caching. Defaults to False.

    Returns:
        subprocess.CompletedProcess: The result of the subprocess run.

    Raises:
        subprocess.CalledProcessError: If the command returns a non-zero exit status.
    """
    # Construct path to Python interpreter in the virtual environment
    if os.name == 'nt':  # Windows
        python_path = os.path.join(venv_path, 'Scripts', 'python.exe')
    else:  # Unix-like systems (Linux, macOS)
        python_path = os.path.join(venv_path, 'bin', 'python')

    command = [
        python_path, "-m", "graphrag.index",
        "--root", root,
        "--reporter", reporter,
        "--emit", emit
    ]
    if config:
        command.append("--config", config)
        
    if verbose:
        command.append("--verbose")
    if resume:
        command.extend(["--resume", resume])
    if nocache:
        command.append("--nocache")

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        return result
    except subprocess.CalledProcessError as e:
        print(f"Command failed with exit code {e.returncode}")
        print(f"Command: {' '.join(e.cmd)}")
        print(f"Error output: {e.stderr}")
        print(f"Standard output: {e.stdout}")
        raise

In [49]:
venv_path = "C:/Users/pdoub/opinionated-rag/.venv"
result = run_graphrag_cli(
    venv_path=venv_path,
    root="./src/graphrag",
    emit="parquet",
    reporter="print"
)
print(result.stdout)


GraphRAG Indexer 
GraphRAG Indexer SUCCESS: Reading settings from src\graphrag\settings.yaml

GraphRAG Indexer INFO: Using default configuration: {
    "llm": {
        "api_key": "REDACTED, length 51",
        "type": "openai_chat",
        "model": "gpt-4o",
        "max_tokens": 4000,
        "request_timeout": 180.0,
        "api_base": null,
        "api_version": null,
        "proxy": null,
        "cognitive_services_endpoint": null,
        "deployment_name": null,
        "model_supports_json": true,
        "tokens_per_minute": 0,
        "requests_per_minute": 0,
        "max_retries": 10,
        "max_retry_wait": 10.0,
        "sleep_on_rate_limit_recommendation": true,
        "concurrent_requests": 25
    },
    "parallelization": {
        "stagger": 0.3,
        "num_threads": 50
    },
    "async_mode": "threaded",
    "root_dir": "./src/graphrag",
    "reporting": {
        "type": "file",
        "base_dir": "output/${timestamp}/reports",
        "storage_account_

___

* Read in data produced by GraphRAG

In [3]:
graph_rag = GraphRAGManager()

Processed 218 entities
Processed 247 relationships
Processed 46 text units
Processed 1 claims
Processed 34 reports


### Example Entities

In [4]:
Markdown(graph_rag.entity_df.sample(5, random_state=4)[['type', 'title', 'description']].to_markdown(index=False))

| type                        | title             | description                                                                                                                                                                                                                                                                                                                                                                                                                              |
|:----------------------------|:------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| "SECURITIES BROKERAGE FIRM" | "MORGAN & MORGAN" | Morgan & Morgan is the law firm representing both Anthony Wright and the appellee Curtis Hamrick.                                                                                                                                                                                                                                                                                                                                        |
| "COURT"                     | "DISTRICT COURTS" | District courts are mentioned as sources of cited cases and are discussed in the context of the balance of power and division of responsibilities with courts of appeal.                                                                                                                                                                                                                                                                 |
| "PLAINTIFF"                 | "PALINDINO"       | "Palindino is mentioned in the context of meeting the elements of crossing state lines and being in the class of transportation industry workers."                                                                                                                                                                                                                                                                                       |
| "LEGAL PROHIBITION"         | "PROTECTIVE CASE" | "Protective cases are lawsuits filed to protect the statute of limitations in various legal contexts."                                                                                                                                                                                                                                                                                                                                   |
| "STATUTE"                   | "SECTION 1"       | Section 1 is a statute that is being interpreted in the context of the Federal Arbitration Act (FAA) case. It contains limiting language regarding the classification of transportation workers. This section plays a crucial role in determining the applicability of the FAA to certain categories of workers, particularly those involved in transportation, by providing specific criteria and limitations for their classification. |

### Example Relationships

In [5]:
Markdown(graph_rag.relationship_df.sample(5, random_state=2)[['source', 'target', 'description']].to_markdown(index=False))

| source                   | target                  | description                                                                                                                                                                                                                                              |
|:-------------------------|:------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| "PLAINTIFFS"             | "SOUTH CAROLINA"        | "Numerous class members from North Carolina physically went into South Carolina every single week."                                                                                                                                                      |
| "STATUTE OF LIMITATIONS" | "BOOTH"                 | "Booth is referenced in the context of filing within the statute of limitations period."                                                                                                                                                                 |
| "HILL"                   | "PALANTINO TEST"        | "Hill case re-adopted the Palantino test."                                                                                                                                                                                                               |
| "APPELLANT"              | "STATUTE LIMITATIONS"   | "The appellant discusses the statute of limitations in the context of the case."                                                                                                                                                                         |
| "PLAINTIFF"              | "STATE ARBITRATION ACT" | The "STATE ARBITRATION ACT" is referenced as the governing law to ascertain whether each "PLAINTIFF" and any opt-in parties should be compelled to individually arbitrate. This determination will be made in accordance with the applicable state laws. |

In [9]:
Markdown(graph_rag.covariate_df.head()[['type', 'subject_id', 'object_id', 'description']].to_markdown(index=False))

| type                   | subject_id     | object_id                  | description                                                                                                                                                                                                                                                                                                                                                                                |
|:-----------------------|:---------------|:---------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| LEGAL DISPUTE          | ANTHONY WRIGHT | WASTE PRO USA INC.         | Anthony Wright filed a consent to join as a named plaintiff in the original South Carolina action in October 2017. The case involves a dispute over whether Mr. Wright's FLSA claim was time-barred and outside the statute of limitations. The district court granted summary judgment against Mr. Wright, ruling that his claim was time-barred. The appeal is to reverse this decision. |
| STATUTE OF LIMITATIONS | ANTHONY WRIGHT | NONE                       | The claim involves a legal argument about whether the statute of limitations was tolled upon the filing of consent by Anthony Wright. The district court ruled that the claim was time-barred, but the appeal argues that the statute of limitations should have been tolled.                                                                                                              |
| LEGAL CONSENT FILING   | MR. WRIGHT     | SOUTH CAROLINA ACTION      | Mr. Wright did file a consent to join in the South Carolina action, which tolls the statute of limitations under the FLSA according to Section 256.                                                                                                                                                                                                                                        |
| COURT JURISDICTION     | WASTEPRO       | DISTRICT OF SOUTH CAROLINA | WastePro raises the question that the District of South Carolina was not a court of competent jurisdiction, which is debated in light of the Supreme Court's Lightfoot decision.                                                                                                                                                                                                           |
|                        |                |                            |                                                                                                                                                                                                                                                                                                                                                                                            |

In [14]:
Markdown(graph_rag.text_unit_df.sample(1, random_state=42)[['n_tokens', 'text']].to_markdown(index=False))

|   n_tokens | text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|-----------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|        520 | lower court held that the arbitration must rise or fall in the application to the FAA because the FAA effectively preempts state arbitration acts. This is wrong for a couple of reasons. First, if the transportation worker exception applies, which is the only circumstance in which the state law argument matters, then that, then that, oh yes, sorry, can I ask you a quick question on this issue? So let&#x27;s assume for the moment that I don&#x27;t agree with you and I agree with the district court on the exception issue. Would I then read the sentence in the arbitration provision, the one that&#x27;s at issue here, a quote that the arbitration provision is governed by the Federal Arbitration Act as unenforceable? Correct. Okay. So if that&#x27;s the case, then if I turn to, this is section 19 of the contract, of the independent worker contract, in the miscellaneous section, subsection C, this is a savings clause where it says, with the exception of the arbitration provision, if any part of this agreement is declared unlawful or unenforceable, the remainder of this agreement shall remain in force in effect. So that&#x27;s a general savings clause, but accepted from that savings clause is the arbitration provision. Don&#x27;t I read that to mean that if any portion of it is unenforceable, then the whole thing falls apart? No, your honor. The savings clause refers to the arbitration agreement as a whole, meaning the parties clearly mutually expressed intent to arbitrate. It&#x27;s the exception, not the limited FAA governed by provision. Yeah, but what the savings clause provision says without the exception part is, if any part of this agreement, any part, is declared unlawful or unenforceable, the remainder shall be in force in effect. And that&#x27;s a very typical savings clause. You don&#x27;t want to lose the contract because some provision is problematic. But the first clause of that is with the exception of the arbitration provision. In general, savings clause doesn&#x27;t apply so that if any part of that provision is unenforceable, then the rest of it shall not remain in force in effect. How do I not read it that way? Because your honor, it doesn&#x27;t say any part of the arbitration provision. It says with the exception of the arbitration provision, which is intending to cover the party&#x27;s mutual intent to arbitrate as a whole, that should |

In [17]:
Markdown(graph_rag.report_df.sample(3, random_state=42)[['community', 'level', 'rank', 'title', 'rank_explanation', 'summary']].to_markdown(index=False))

|   community |   level |   rank | title                                                 | rank_explanation                                                                                                                                      | summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|------------:|--------:|-------:|:------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|           3 |       0 |    7.5 | Statute and Legal Interpretations                     | The impact severity rating is high due to the significant legal implications and the involvement of multiple high-level legal entities and cases.     | The community revolves around the 'STATUTE' entity, which is central to various legal provisions and interpretations. Key entities include Congress, Section 1, and several court cases such as the Wayne case and Bailey. The relationships among these entities highlight the intricate legal framework and the roles of different courts in interpreting statutes.                                                                                                                                     |
|          33 |       1 |    8.5 | New Prime v. Oliveira and the Federal Arbitration Act | The impact severity rating is high due to the far-reaching legal implications of the Supreme Court decision on employment and arbitration agreements. | The community revolves around the landmark Supreme Court case New Prime v. Oliveira, which addresses the classification of independent contractor agreements under the Federal Arbitration Act (FAA). Key entities include the Transportation Worker Exemption, the First Circuit, and TIA Final Mile. The relationships among these entities highlight the significant legal implications of the case, particularly in the context of arbitration and employment law.                                    |
|          34 |       1 |    7.5 | Mr. Morgan and the Contract of Employment Case        | The impact severity rating is high due to the potential legal precedents and implications for the transportation and delivery industry.               | The community revolves around Mr. Morgan, who is a key defendant in a legal case concerning the contract of employment and the Georgia Arbitration Code. The case involves multiple entities including the district court, the Olivera case, and Justice Gorsuch, all of which are interconnected through the central issue of whether an independent contractor agreement qualifies as a contract of employment. The court proceedings and the interpretations of the law are pivotal in this community. |

In [19]:
Markdown(graph_rag.report_df['full_content'].tolist()[0])

# District Court and Arbitration Cases

The community revolves around the District Court, which is involved in numerous cases related to arbitration and employment contracts. Key entities include the District Court, various legal cases such as New Prime v. Oliveira, and significant rulings like the Hill standard. The relationships between these entities highlight the ongoing legal debates and the court's role in interpreting and applying arbitration agreements and employment laws.

## District Court's central role in arbitration and employment cases

The District Court is a pivotal entity in this community, handling numerous cases involving arbitration and employment contracts. The court's decisions have far-reaching implications, particularly in interpreting the Federal Arbitration Act and state arbitration codes. The District Court's rulings on the arbitrability of disputes and the nature of employment contracts are critical in shaping legal standards and precedents [Data: Entities (51); Relationships (87, 112, 116, 122, 123, +more)].

## Significance of the New Prime v. Oliveira decision

The New Prime v. Oliveira decision is a landmark case referenced by the District Court. This Supreme Court ruling addresses the issue of arbitrability and has significant implications for the District Court's ongoing cases. The court is required to reexamine its decisions in light of this ruling, which underscores the importance of the New Prime case in shaping legal interpretations of arbitration agreements [Data: Entities (138, 141); Relationships (106, 107, 114)].

## Application of the Hill standard

The District Court applied the Hill standard to determine whether workers were engaged in the flow of interstate commerce. This standard is crucial in deciding the applicability of certain legal provisions, particularly in cases involving transportation workers. The court's interpretation of the Hill standard has significant implications for the classification of workers and the enforcement of arbitration agreements [Data: Entities (180); Relationships (117, 120)].

## Impact of the Georgia Arbitration Code

The District Court addressed arguments related to the Georgia Arbitration Code, ultimately deeming them untimely. This decision highlights the procedural aspects of arbitration cases and the importance of timely arguments in legal proceedings. The court's ruling on the Georgia Arbitration Code reflects its approach to state-specific arbitration laws and their interaction with federal standards [Data: Entities (140); Relationships (110)].

## Role of Judge Thrash in factual findings

Judge Thrash played a significant role in making factual findings at the District Court level. These findings are critical in shaping the court's decisions on arbitration and employment contracts. Judge Thrash's rulings provide a foundation for the court's legal interpretations and the application of relevant standards and precedents [Data: Entities (104); Relationships (105)].

## Influence of supplemental authorities like Gates v. TF Final Mile

The District Court referenced the Gates v. TF Final Mile case as a supplemental authority in its rulings. This case involves the transportation worker exemption and arbitration agreements, aligning with similar judicial interpretations in the same jurisdiction. The use of supplemental authorities underscores the interconnectedness of legal cases and the importance of precedent in judicial decision-making [Data: Entities (198); Relationships (123)].

## Challenges to the District Court's interpretations

The District Court's interpretations and rulings are subject to challenges and appeals. For instance, the Court of Appeals for the Eleventh Circuit is reviewing the District Court's order compelling arbitration. This appellate review process highlights the dynamic nature of legal interpretations and the ongoing scrutiny of lower court decisions by higher courts [Data: Relationships (3, 5)].

## District Court's handling of employment contract issues

The District Court is responsible for making decisions regarding employment contracts, particularly in the context of independent contractor agreements. The court's findings on the nature of these contracts and their implications for arbitration are crucial in determining the rights and obligations of the parties involved. These decisions have significant legal and practical implications for employment relationships [Data: Relationships (115, 108)].

## Run Some Test Queries

# Query 1 - Global Search
"Summarize the court's opinion of Liberty Mutual's handling of coverage denials"

In [22]:
result = await graph_rag.search_global("Summarize the arguments presented by Ryan Morgan.")
Markdown(result.response)

# Summary of Arguments Presented by Ryan Morgan

## Introduction
Ryan Morgan, a prominent attorney, is involved in several high-profile appellate cases, representing clients such as Anthony Wright, Curtis Hamrick, and Bruce Gates. His arguments span various legal issues, including the Fair Labor Standards Act (FLSA), the Georgia Arbitration Code, and the transportation worker exception. This report synthesizes the key points and implications of Morgan's arguments based on the available data.

## Key Arguments

### Statute of Limitations and FLSA Claims
Morgan argues that the statute of limitations for Anthony Wright's FLSA claim should be tolled upon the filing of consent, as specified in 29 U.S.C. 256. This argument is central to Wright's appeal against the district court's summary judgment ruling that his claim was time-barred [Data: Reports (28)].

### Interpretation of the Georgia Arbitration Code
Morgan raises significant issues regarding the interpretation of the transportation worker exception and the arbitration agreement under the Georgia Arbitration Code. He contends that an independent contractor agreement can qualify as a contract of employment, referencing the Olivera case to support this proposition [Data: Reports (34, +more)].

### Representation of Multiple Clients
Morgan's representation of multiple clients, including Anthony Wright, Curtis Hamrick, and Bruce Gates, highlights his extensive role in advocating for clients in complex legal situations. His involvement in these cases underscores his ability to handle a variety of legal issues and his commitment to protecting his clients' rights [Data: Reports (28, +more)].

### Legal Strategy and Statutory Interpretation
Morgan's legal strategy involves referencing specific legal statutes and precedents to support his clients' claims. For instance, his use of 29 U.S.C. 256 in Wright's case underscores the importance of statutory interpretation in his arguments. This approach is pivotal in shaping the court's understanding of the legal issues at hand [Data: Reports (28, +more)].

### Transportation and Delivery Industry
Morgan addresses the court on matters related to the transportation and delivery industry, particularly in the context of the transportation worker exception and the arbitration agreement. His arguments are crucial in determining whether an independent contractor agreement qualifies as a contract of employment under the Georgia Arbitration Code [Data: Reports (34, +more)].

## Conclusion
Ryan Morgan's arguments are central to the ongoing court proceedings involving his clients. By focusing on statutory interpretation, the transportation worker exception, and the arbitration agreement, Morgan aims to protect his clients' rights and ensure a fair legal process. His involvement in these cases demonstrates his expertise in handling complex legal issues and his commitment to advocating for his clients.

---

This report synthesizes the key points and implications of Ryan Morgan's arguments based on the available data. For further details, please refer to the specific data reports mentioned.

## Query 2 - Local Search
"How has Liberty Mutual's use of Code x553 influenced the legal proceedings?"

In [23]:
result = await graph_rag.search_local("What specific legal statutes and precedents did Ryan Morgan use in his arguments?")
Markdown(result.response)

I'm sorry, but I don't have the information regarding the specific legal statutes and precedents used by Ryan Morgan in his arguments. If you can provide the relevant data tables or additional context, I would be happy to help analyze and summarize the information for you.

## Query 3 - Local Search
"How do the coverage rulings specifically impact Liberty Mutual? What risks (financial or process-related) exist and how can the carrier best mitigate them?"

In [67]:
result = await graph_rag.search_local(
    "How do the coverage rulings specifically impact Liberty Mutual? What risks (financial or process-related) exist and how can the carrier best mitigate them?"
)
Markdown(result.response)

# Impact of Coverage Rulings on Liberty Mutual

## Overview

Liberty Mutual is currently navigating a complex legal landscape shaped by various coverage rulings. These rulings have significant implications for the company's financial stability, operational processes, and legal strategies. This report delves into the specific impacts of these rulings, the associated risks, and the best practices for mitigating these risks.

## Specific Impacts of Coverage Rulings

### Interpretation of 'Direct Physical Loss'

One of the most critical coverage rulings involves the interpretation of 'direct physical loss.' Courts have generally ruled that COVID-19 does not constitute 'direct physical loss,' which limits the scope of claims Liberty Mutual may need to cover. This precedent could significantly reduce the number of pandemic-related claims the company must pay out, thereby mitigating immediate financial exposure [Data: Reports (156, 165, 163, 160, 166)].

### Uninsured Motorist (UM) Coverage

The ruling in 'Thounsavath, 2018 IL 122558' reinforces that if an individual is considered insured for liability purposes, they automatically qualify for UM coverage. This decision prevents insurance companies from denying UM coverage based on conflicting policy terms, thereby upholding statutory protections for insured individuals. This ruling impacts how Liberty Mutual's policies are interpreted and enforced, potentially leading to increased payouts and higher reserves for potential claims [Data: Reports (185, 180, 182)].

### Delaware Workers' Compensation Law

Liberty Mutual is involved in a multifaceted legal dispute concerning the use of billing Code X553 under Delaware Workers' Compensation Law. The outcomes of these legal battles could set important precedents for how insurance denials are handled and the necessary compliance with statutory requirements. This practice could lead to legal challenges, highlighting the importance of these documents in the operational and compliance landscape of the group [Data: Reports (143, 142, 34)].

### Interpretation of Policy Terms

The coverage rulings also impact Liberty Mutual by setting legal precedents that could influence how similar insurance policies are interpreted in the future. This includes the interpretation of terms like 'sporting equipment' and 'fixtures,' which have broad implications for coverage claims and liability [Data: Reports (227, 239, 238, 236, 233)].

### Jurisdictional Implications

The Superior Court's decision in Mills Ltd. Partnership v. Liberty Mutual Insurance Co. established that Delaware has the most significant relationship with the risks insured by a D&O policy. This ruling impacts Liberty Mutual by setting a precedent for how D&O policies are interpreted in future legal disputes, emphasizing the importance of jurisdiction in legal determinations [Data: Reports (276)].

## Financial and Process-Related Risks

### Financial Risks

1. **Increased Litigation Costs and Claim Payouts**: Financial risks for Liberty Mutual include potential increases in litigation costs and claim payouts if courts were to rule differently in future cases. The current rulings favoring insurers in denying COVID-19 related claims help mitigate immediate financial exposure, but ongoing legal battles could alter this landscape [Data: Reports (156, 165, 163, 160, 166)].

2. **Potential Liabilities from UM Coverage**: Broader interpretations of UM coverage could lead to increased payouts and higher reserves for potential claims. This could significantly impact the company's financial stability [Data: Reports (180, 182, 185)].

3. **Liabilities from Workers' Compensation Claims**: The controversy surrounding 'Code x553' and its alleged contravention of sections of the Delaware Workers’ Compensation Code raises questions about the legality and ethicality of such denials. This could have broader implications for Liberty Mutual’s reputation and regulatory compliance [Data: Reports (34)].

### Process-Related Risks

1. **Need for Policy Language Updates**: Process-related risks involve the need for Liberty Mutual to continuously update and align their policy language with prevailing legal interpretations to avoid ambiguities that could lead to disputes. Ensuring clear definitions and exclusions in policy terms is crucial to mitigate these risks [Data: Reports (156, 165, 163, 160, 166)].

2. **Compliance with Judicial Interpretations**: The need to update policy language and claims processing procedures to ensure compliance with judicial interpretations and statutory requirements is essential. Failure to do so could result in legal challenges and penalties [Data: Reports (180, 182, 185)].

3. **Jurisdictional Considerations**: The ruling underscores the need for Liberty Mutual to carefully consider the choice-of-law provisions in their policies, as Delaware law will likely be applied in disputes involving Delaware-incorporated companies. This could affect the interpretation of policy terms and the outcome of coverage disputes [Data: Reports (276)].

## Mitigation Strategies

### Regular Policy Reviews and Updates

To mitigate these risks, Liberty Mutual should regularly review and update their policy language to align with current legal standards and judicial interpretations. This includes adopting clearer and more precise language in their insurance policies and utilizing authoritative sources like dictionaries and legal texts for defining terms [Data: Reports (180, 182, 185, 243, 230, 242)].

### Comprehensive Training Programs

Implementing comprehensive training programs for claims adjusters on the latest legal requirements and best practices can help ensure compliance and reduce the risk of legal disputes. This proactive approach can help in minimizing potential liabilities and improving operational efficiency [Data: Reports (180, 182, 185)].

### Enhanced Communication and Documentation

Liberty Mutual can also mitigate risks by enhancing communication with policyholders about coverage limits and exclusions, particularly in relation to pandemic-related claims. Clear and transparent communication can reduce the likelihood of disputes and improve customer trust [Data: Reports (156, 165, 163, 160, 166)].

### Proactive Legal Strategies

Engaging in proactive legal reviews and consultations to stay ahead of potential legal challenges is crucial. This includes closely monitoring legal developments and court rulings related to insurance coverage to stay informed about emerging legal trends and adjust policies and practices accordingly [Data: Reports (227, 239, 244, 245)].

### Internal Audit Systems

Implementing a robust internal audit system to regularly review and update billing codes and practices could help in maintaining compliance and avoiding future legal challenges. Ensuring that all administrative practices align with legal and regulatory requirements is essential for mitigating process-related risks [Data: Reports (32, 34)].

## Conclusion

The coverage rulings have far-reaching implications for Liberty Mutual, impacting both financial stability and operational processes. By adopting a proactive approach to policy reviews, training, communication, and legal strategies, Liberty Mutual can effectively mitigate these risks and navigate the complex legal landscape.

### Example of 'sources'

In [68]:
reports_df = result.context_data["reports"]

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,309,# FSO vs. Liberty: Legal Dispute Over Workers'...,3,7.5,FSO vs. Liberty: Legal Dispute Over Workers' C...,The impact severity rating is high due to the ...,The community centers around First State Ortho...,[{'explanation': 'FSO has initiated a legal ba...,"{\n ""title"": ""FSO vs. Liberty: Legal Disput...",761d42b8-4215-462e-a73e-5da4f20c7d1e
1,310,# Legal Disputes Involving Insurance Claims an...,3,7.5,Legal Disputes Involving Insurance Claims and ...,The impact severity rating is relatively high ...,This report focuses on a series of legal compl...,"[{'explanation': 'First State Orthopaedics, P....","{\n ""title"": ""Legal Disputes Involving Insu...",f56048fe-b91f-4784-a9a7-50c2cf8271c4
2,311,# Liberty and Code X553 Legal Dispute\n\nThe c...,3,7.5,Liberty and Code X553 Legal Dispute,The impact severity rating is high due to the ...,"The community centers around Liberty, an insur...",[{'explanation': 'Liberty has been actively de...,"{\n ""title"": ""Liberty and Code X553 Legal D...",6afaf0e9-111d-4260-8338-1ac52a9e6717
3,312,# Section 2322F(e) and Coverage Denial Interpr...,3,7.5,Section 2322F(e) and Coverage Denial Interpret...,The impact severity rating is relatively high ...,This report focuses on the legal interpretatio...,[{'explanation': 'Section 2322F(e) is pivotal ...,"{\n ""title"": ""Section 2322F(e) and Coverage...",69b7d134-68bf-4e40-9917-4b24ffc6086e
4,313,# Hosanna vs. Hartford: Theatrical Insurance D...,3,7.5,Hosanna vs. Hartford: Theatrical Insurance Dis...,The impact severity rating is relatively high ...,This report examines the complex legal and ins...,[{'explanation': 'Hosanna is engaged in a lega...,"{\n ""title"": ""Hosanna vs. Hartford: Theatri...",a8f4a61c-686f-4ae5-b0cf-e784943dabb4
...,...,...,...,...,...,...,...,...,...,...
331,13,# Superior Court and RSUI: Legal Dynamics in I...,0,8.5,Superior Court and RSUI: Legal Dynamics in Ins...,The impact severity rating is high due to the ...,This report delves into the intricate legal in...,[{'explanation': 'The Superior Court plays a p...,"{\n ""title"": ""Superior Court and RSUI: Lega...",211c8749-86fd-4c91-b1c1-5c3206178e6d
332,2,# Hartford and Theatrical Insurance Disputes\n...,0,7.5,Hartford and Theatrical Insurance Disputes,The impact severity rating is relatively high ...,This report examines the complex legal interac...,[{'explanation': 'The interpretation of 'direc...,"{\n ""title"": ""Hartford and Theatrical Insur...",70287752-8d64-4412-82d2-98ebd03794b4
333,3,# Direct Auto Insurance Co. and Uninsured Moto...,0,7.5,Direct Auto Insurance Co. and Uninsured Motori...,The impact severity rating is high due to the ...,This report focuses on the legal disputes invo...,[{'explanation': 'Direct Auto Insurance Co. is...,"{\n ""title"": ""Direct Auto Insurance Co. and...",7310a69d-b009-4fbb-aad1-9a583a15899a
334,7,# United Behavioral Health and ERISA Complianc...,0,8.5,United Behavioral Health and ERISA Compliance ...,The impact severity rating is high due to the ...,This report delves into the complex legal land...,[{'explanation': 'United Behavioral Health (UB...,"{\n ""title"": ""United Behavioral Health and ...",ac519157-fa03-40f6-be63-040e86dbd486


### Community Summary Example

In [71]:
Markdown(report_df[report_df["community"]=='34']["full_content"].tolist()[0])

# Liberty Mutual's Group and Explanation of Benefits Forms

This report examines the relationships and activities within Liberty Mutual’s Group, focusing on the use of 'Code x553' in Explanation of Benefits forms to deny workers' compensation claims. Mutual Holding Company, Inc. plays a significant role by providing third-party claims administration services to the group.

## Explanation of Benefits Forms and their impact

The Explanation of Benefits forms issued by Liberty Mutual’s Group using 'Code x553' have been a central element in denying coverage for workers’ compensation bills. This practice has significant implications for the financial and legal standing of the entity, as it directly affects the compensation of workers. The use of specific codes to deny claims could potentially lead to legal challenges, highlighting the importance of these documents in the operational and compliance landscape of the group [Data: Entities (1625), Relationships (2461, 2462, 2460)].

## Role of Mutual Holding Company, Inc.

Mutual Holding Company, Inc. provides crucial administrative services to Liberty Mutual’s Group, which includes the issuance of the controversial Explanation of Benefits forms. This entity’s role is pivotal in the administration of claims and potentially influences the group’s compliance with regulatory standards. The relationship between Mutual Holding Company, Inc. and Liberty Mutual’s Group underscores the interconnected nature of their operations and the potential risks associated with their administrative practices [Data: Entities (1623, 1624), Relationships (2459)].

## Controversy surrounding 'Code x553'

The use of 'Code x553' in the Explanation of Benefits forms has been a point of contention, particularly with FSO arguing that this practice contravenes sections of the Delaware Workers’ Compensation Code. This dispute not only highlights the specific operational practices of Liberty Mutual’s Group but also raises questions about the legality and ethicality of such denials. The controversy could have broader implications for the group’s reputation and regulatory compliance [Data: Relationships (2461, 2462)].

### Context Entities Example

In [79]:
Markdown(relationship_df[relationship_df["human_readable_id"].isin(['2461', '2462'])][['source', 'target', 'description', 'human_readable_id']].to_markdown())

|      | source                          | target      | description                                                                                                                                            |   human_readable_id |
|-----:|:--------------------------------|:------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------:|
| 2461 | "EXPLANATION OF BENEFITS FORMS" | "FSO"       | "FSO argues that the denial of coverage in the Explanation of Benefits forms contravenes certain sections of the Delaware Workers’ Compensation Code." |                2461 |
| 2462 | "EXPLANATION OF BENEFITS FORMS" | "CODE X553" | "Code x553 is used in the Explanation of Benefits forms to deny coverage of workers’ compensation bills."                                              |                2462 |

In [82]:
relationship_df[relationship_df["human_readable_id"].isin(['2461', '2462'])]['text_unit_ids']

2461    [60937e9dc6e07123c03d887841bd5ffd]
2462    [60937e9dc6e07123c03d887841bd5ffd]
Name: text_unit_ids, dtype: object

## Original Text

In [85]:
Markdown(text_unit_df[text_unit_df["id"]=="60937e9dc6e07123c03d887841bd5ffd"]["text"].tolist()[0])

 Mutual Holding Company, Inc., providing third-party claims

administration services to Liberty Mutual’s Group members, including the other

Appellants.5 Collectively, Appellants are referred to as “Liberty.”

          In 2017 and 2018, Liberty routinely issued Explanation of Benefits forms to

FSO using “Code x553.” Those forms purported to deny coverage of workers’

compensation bills for procedures that FSO performed.6            The Explanation of

Benefits generated by Code x553 stated:

          THIS SERVICE NOT AUTHORIZED BY CASE MANAGER.
          PLEASE CONTACT THE CASE MANAGER FOR FURTHER
          INFORMATION.7

          FSO argues that this denial contravenes certain sections of the Delaware

Workers’ Compensation Code because the response fails to provide a meaningful

explanation for the reason for the denial.8 FSO relies on three sections of Title 19 to

support its position. First, 19 Del. C. § 2362(b) mandates that


3
    Id. at A368.
4
    Id.
5
    Id. at A368–69.
6
    Id. at A370.
7
    Id.
8
    Id. at A369–70.

                                           3
           [a]ll medical expenses shall be paid within 30 days after bills and
           documentation for said expenses are received by the employer or its
           insurance carrier for payment, unless the carrier or self

## Cost for the above 3 queries

In [64]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 12. LLM tokens: 134489
